# Appear Here Test

Pauline Guilmin

January 2019

# import libraries

In [323]:
import numpy as np 
import pandas as pd
pd.options.display.max_colwidth = 800

In [324]:
!pwd

/Users/paguilmin/Documents/AppearHere_Test/analysis


# Charge data

In [325]:
path = '../data/'
ideas = pd.read_csv(path+'ideas.csv',sep=',')
transitions = pd.read_csv(path+'transitions.csv',sep=',')

## cleaning

In [326]:
ideas = ideas.drop(['Unnamed: 0'],axis=1)
transitions.columns
transitions = transitions.drop(['Unnamed: 0'],axis=1)

In [327]:
total.loc[total['start_on'].str.startswith('2')==False,['created_at','start_on','end_on']]
#'|'.join(['2018','2019','2020']

transitions.loc[transitions['start_on'].str.startswith('2')==False,'start_on'] = \
transitions.loc[transitions['start_on'].str.startswith('2')==False,'start_on'].str.replace('0019','2019')
transitions.loc[transitions['end_on'].str.startswith('2')==False,'end_on'] = \
transitions.loc[transitions['end_on'].str.startswith('2')==False,'end_on'].str.replace('0020','2020')
transitions.loc[transitions['end_on'].str.startswith('2')==False,'end_on'] = \
transitions.loc[transitions['end_on'].str.startswith('2')==False,'end_on'].str.replace('0019','2019')

In [328]:
transitions.loc[:,'transition_ts'] = pd.to_datetime(transitions.loc[:,'transition_ts'])
transitions.loc[:,'start_on'] = pd.to_datetime(transitions.loc[:,'start_on'])
transitions.loc[:,'end_on'] = pd.to_datetime(transitions.loc[:,'end_on'])

ideas.loc[:,'created_at'] = pd.to_datetime(ideas.loc[:,'created_at'])

# useful fonctions

In [329]:
def top_champ(df,feature,number) :
    df_ok = df.copy()
    df_ok[feature] = df_ok[feature].fillna('unknown')
    top_10 = df_ok.groupby(feature).agg('size').sort_values(ascending=False).to_frame().reset_index()
    top_10.columns=[str(x) for x in feature]+['nb_records']
    top_10['percentage']=(100.* (top_10['nb_records'] / top_10['nb_records'].sum())).round(2)
    print('%0.0f distinct values for the feature %s' %(top_10.shape[0],feature))
    return top_10.head(number)

# ideas exploration

In [330]:
ideas.nunique()

created_at    4000
category         6
idea_id       4000
dtype: int64

In [331]:
top_champ(ideas,['category'],10)

6 distinct values for the feature ['category']


,category,nb_records,percentage
0,popup_shop,2174,54.35
1,private_event,546,13.65
2,popup_bar,533,13.32
3,unique,398,9.95
4,gallery,282,7.05
5,sample_sale,67,1.68


In [332]:
min_created = min(ideas['created_at'])
max_created = max(ideas['created_at'])
print('Ideas have been created from %s to %s' %(min_created, max_created))

Ideas have been created from 2018-01-01 17:01:01.489000 to 2019-01-21 10:19:35.205000


# transitions exploration

In [333]:
transitions.nunique()

transition_ts         21062
category                  6
idea_id                2677
booking_request_id     9942
to_state                  4
space_id               2065
country                   3
city                    110
portfolio_type            9
floor_size              548
floor_size_unit           2
start_on                503
end_on                  618
price                  3593
dtype: int64

In [334]:
transitions = transitions.sort_values(['idea_id','space_id','transition_ts'])

In [335]:
transitions['to_state_bf']= transitions['to_state'].shift(1)
transitions['to_state_af']= transitions['to_state'].shift(-1)
transitions['time_booked'] = transitions.loc[:,'end_on']- transitions.loc[:,'start_on']
transitions.loc[:,'time_booked'] = transitions.loc[:,'time_booked']/ np.timedelta64(1, 'D')

In [336]:
id_bk_notime = transitions.loc[transitions['time_booked']==0,'booking_request_id'].unique()

In [361]:
168132 in id_bk_notime

False

In [360]:
transitions[transitions['booking_request_id']==168132.0]

,transition_ts,category,idea_id,booking_request_id,to_state,space_id,country,city,portfolio_type,floor_size,floor_size_unit,start_on,end_on,price,to_state_bf,to_state_af,time_booked
463,2018-10-24 13:51:03.343,popup_shop,105,168132,booked,5562,GB,London,4.0,10,0,2018-12-07,2018-12-14,12000,declined,booked,7.0


In [337]:
states_ideas = transitions.groupby(['idea_id','to_state']).agg('size').to_frame().reset_index()
idea_id_booked = states_ideas.loc[states_ideas['to_state']=='booked','idea_id'].unique()
print('Above 4000 ideas %0.0f ideas have been booked at least one time' %len(idea_id_booked))
#states_ideas[states_ideas['idea_id'].isin(idea_id_booked)]


Above 4000 ideas 328 ideas have been booked at least one time


In [340]:
top_champ(transitions[transitions['to_state']=='matched'],['to_state_af'],10)

4 distinct values for the feature ['to_state_af']


,to_state_af,nb_records,percentage
0,declined,2125,78.88
1,booked,388,14.40
2,submitted,117,4.34
3,matched,64,2.38


In [341]:
transitions.loc[(transitions['to_state']=='booked') &
           (transitions['to_state_bf']=='declined'),'idea_id'].unique()

array([ 105,  230,  245,  323,  430,  449,  451,  834,  902,  917, 1094,
       1154, 1335, 1535, 1692, 1698, 1794, 1919, 2161, 2214, 2789, 2846,
       3034, 3183, 3329, 3510, 3791, 3861, 3896, 3926])

In [342]:
#transitions[transitions['space_id']==6735]
#transitions[transitions['idea_id']==105]

# total exploration

In [343]:
total = pd.merge(ideas, transitions, how='left', left_on='idea_id',right_on='idea_id')

In [344]:
total.loc[:,'time_after_creation'] = total.loc[:,'transition_ts']- total.loc[:,'created_at']
total.loc[:,'time_after_creation'] = total.loc[:,'time_after_creation']/ np.timedelta64(1, 'D')
total_first = total.drop_duplicates('idea_id')
top_champ(total_first,['to_state'],10)

4 distinct values for the feature ['to_state']


,to_state,nb_records,percentage
0,submitted,2377,59.42
1,unknown,1323,33.07
2,declined,295,7.38
3,booked,5,0.12


In [345]:
total_first.loc[:,'time_after_creation'].describe()
print(total[total['category_x']!=total['category_y']].shape)
print(total[total['category_y'].isnull()].shape)
#idem
# 1323 ideas without 

(1323, 20)
(1323, 20)


In [346]:
idea_id_unknown = total.loc[total['category_y'].isnull(),'idea_id'].unique()
min_created_ukn = min(ideas.loc[ideas['idea_id'].isin(idea_id_unknown),'created_at'])
max_created_ukn = max(ideas.loc[ideas['idea_id'].isin(idea_id_unknown),'created_at'])
print('There are %0.0f ideas unknown' %len(idea_id_unknown))
print('Ideas unknown have been created from %s to %s' %(min_created_ukn, max_created_ukn))
idea_id_known = total.loc[total['category_y'].notnull(),'idea_id'].unique()
min_created_ukn = min(ideas.loc[ideas['idea_id'].isin(idea_id_known),'created_at'])
max_created_ukn = max(ideas.loc[ideas['idea_id'].isin(idea_id_known),'created_at'])
print('There are %0.0f ideas with at least one request submitted' %(len(idea_id_known)))
print('Ideas with requests have been created from %s to %s' %(min_created_ukn, max_created_ukn))

There are 1323 ideas unknown
Ideas unknown have been created from 2018-01-01 17:01:01.489000 to 2019-01-21 10:19:35.205000
There are 2677 ideas with at least one request submitted
Ideas with requests have been created from 2018-01-02 15:21:32.044000 to 2019-01-15 14:54:49.719000


In [347]:
top_champ(ideas[ideas['idea_id'].isin(idea_id_unknown)],['category'],10)

6 distinct values for the feature ['category']


,category,nb_records,percentage
0,popup_shop,715,54.04
1,popup_bar,234,17.69
2,unique,142,10.73
3,private_event,134,10.13
4,gallery,76,5.74
5,sample_sale,22,1.66


In [348]:
top_champ(ideas[ideas['idea_id'].isin(idea_id_known)],['category'],10)

6 distinct values for the feature ['category']


,category,nb_records,percentage
0,popup_shop,1459,54.50
1,private_event,412,15.39
2,popup_bar,299,11.17
3,unique,256,9.56
4,gallery,206,7.70
5,sample_sale,45,1.68


In [349]:
total_ok = total.loc[total['idea_id'].isin(idea_id_known)]

In [350]:
total_ok['time_after_creation'].describe()

count    2.106200e+04
mean     2.365659e+01
std      4.890799e+01
min      6.250000e-07
25%      1.163474e-01
50%      4.105958e+00
75%      2.042294e+01
max      3.599833e+02
Name: time_after_creation, dtype: float64

In [351]:
total_ok.loc[total_ok['time_after_creation']>15,'after_creation']='sup_15j'
total_ok.loc[total_ok['time_after_creation']<=15,'after_creation']='3j_15j'
total_ok.loc[total_ok['time_after_creation']<3,'after_creation']='1j_3j'
total_ok.loc[total_ok['time_after_creation']<1,'after_creation']='1h_1j'
total_ok.loc[total_ok['time_after_creation']<0.04,'after_creation']='10s_1h'
total_ok.loc[total_ok['time_after_creation']<0.0001,'after_creation']='less_10s'

In [352]:
top_champ(total_ok.drop_duplicates('idea_id'),['after_creation'],10)
# More than an half of the ideas transforms in a request in less than 10 seconds

6 distinct values for the feature ['after_creation']


,after_creation,nb_records,percentage
0,less_10s,1556,58.12
1,sup_15j,282,10.53
2,3j_15j,280,10.46
3,10s_1h,240,8.97
4,1h_1j,202,7.55
5,1j_3j,117,4.37


In [354]:
total_ok[['transition_ts','idea_id','space_id','to_state']][0:10]

,transition_ts,idea_id,space_id,to_state
2,2018-10-08 19:38:18.153,2,366.0,submitted
3,2018-10-10 12:19:26.197,2,366.0,declined
5,2018-06-10 16:05:47.226,4,5844.0,submitted
6,2018-06-26 15:55:27.471,4,5844.0,declined
7,2018-11-27 17:06:18.287,5,5241.0,submitted
8,2018-11-28 16:46:10.694,5,5241.0,matched
9,2018-12-01 01:03:33.476,5,5241.0,booked
10,2018-03-27 14:46:16.156,6,4035.0,submitted
11,2018-04-10 17:27:06.135,6,4035.0,declined
12,2018-03-27 14:46:16.313,6,5438.0,submitted


In [447]:
conversion = total_ok.groupby(['idea_id','space_id','booking_request_id'])['to_state'].\
apply(lambda x: "%s" % ' / '.join(x)).to_frame().reset_index()
#conversion
#top_champ(conversion,['to_state'],10)
conversion.loc[:,'ending_stage'] = conversion.loc[:,'to_state'].apply(lambda x : x.split(' ')[-1])

conversion_2 = conversion\
.join(conversion['to_state'].str.split(' / s', expand=True)\
      .stack().reset_index(level=1, drop=True).rename('to_state_2'))
conversion_2.loc[:,'to_state_2'] = conversion_2['to_state_2'].str.replace('ubmitted','submitted')
conversion_2.loc[:,'to_state_2'] = conversion_2['to_state_2'].str.replace('ssubmitted','submitted')
conversion_2.loc[:,'ending_stage_2'] = conversion_2.loc[:,'to_state_2'].apply(lambda x : x.split(' ')[-1])



In [451]:
top_champ(conversion,['ending_stage'],100)

4 distinct values for the feature ['ending_stage']


,ending_stage,nb_records,percentage
0,declined,9074,91.27
1,booked,456,4.59
2,submitted,329,3.31
3,matched,83,0.83


In [471]:
from operator import itemgetter
from itertools import groupby


In [476]:
#conversion['to_state_simpl'] = conversion['to_state'].str.split(' / ').apply(lambda x : list(map(itemgetter(0), groupby(x))))
#conversion['to_state_simpl']

In [479]:
top_champ(conversion[conversion.ending_stage=='submitted'],['to_state'],100)

4 distinct values for the feature ['to_state']


,to_state,nb_records,percentage
0,submitted,315,95.74
1,submitted / submitted,8,2.43
2,submitted / declined / submitted,5,1.52
3,submitted / submitted / submitted,1,0.30


In [500]:
#conversion[(conversion['to_state']=='submitted / declined / submitted / declined')]
#conversion[conversion['idea_id']==30]

In [444]:
print(transitions.drop_duplicates(['booking_request_id','city','space_id']).shape)
transitions.drop_duplicates(['booking_request_id']).shape

(9942, 17)


(9942, 17)

In [460]:
test = total_ok[(total_ok['booking_request_id']==114598.0)]
test.iloc[:,10:20]\
#[['created_at','transition_ts','booking_request_id','space_id','to_state','start_on','end_on','time_booked']]

,portfolio_type,floor_size,floor_size_unit,start_on,end_on,price,to_state_bf,to_state_af,time_booked,time_after_creation
232,3.0,1000.0,0.0,2018-06-11,2018-06-17,500000.0,declined,declined,6.0,20.862976
233,3.0,1000.0,0.0,2018-06-11,2018-06-17,500000.0,submitted,submitted,6.0,20.993371
234,3.0,1000.0,0.0,2018-06-11,2018-06-17,500000.0,declined,declined,6.0,21.760294
235,3.0,1000.0,0.0,2018-06-11,2018-06-17,500000.0,submitted,declined,6.0,43.741980


In [401]:
test = total_ok[(total_ok['booking_request_id']==146438.0)]
test.iloc[:,10:20]

,portfolio_type,floor_size,floor_size_unit,start_on,end_on,price,to_state_bf,to_state_af,time_booked,time_after_creation
130,8.0,600.0,0.0,2018-09-28,2018-10-01,57140.0,declined,submitted,3.0,0.000001
131,8.0,600.0,0.0,2018-09-28,2018-10-01,57140.0,submitted,declined,3.0,0.968679
132,8.0,600.0,0.0,2018-09-28,2018-10-01,57140.0,submitted,declined,3.0,5.098823


,portfolio_type,floor_size,floor_size_unit,start_on,end_on,price,to_state_bf,to_state_af,time_booked,time_after_creation
130,8.0,600.0,0.0,2018-09-28,2018-10-01,57140.0,declined,submitted,3.0,0.000001
131,8.0,600.0,0.0,2018-09-28,2018-10-01,57140.0,submitted,declined,3.0,0.968679
132,8.0,600.0,0.0,2018-09-28,2018-10-01,57140.0,submitted,declined,3.0,5.098823


In [255]:
test = total_ok.groupby(['idea_id','space_id','booking_request_id']).agg('size').to_frame().reset_index()
test.shape

(9942, 4)

In [257]:
test.nunique()

idea_id               2677
space_id              2065
booking_request_id    9942
0                        7
dtype: int64

# studies of bookings ok

In [502]:
bookings_ok = pd.merge(conversion[['booking_request_id','to_state','ending_stage']],
                      total_ok.drop_duplicates('booking_request_id')[['idea_id','booking_request_id','space_id','country', 'city',
       'portfolio_type', 'floor_size', 'floor_size_unit','time_booked',
       'time_after_creation', 'after_creation']],how='inner', on = 'booking_request_id')


In [503]:
# reservation more than 1 day (without bug)
bookings_ok =bookings_ok[bookings_ok['time_booked']>0]

In [504]:
# finished bookings : either declined or booking
bookings_ok = bookings_ok[bookings_ok['ending_stage'].isin(['declined','booked'])]

In [505]:
top_champ(bookings_ok,['ending_stage'],100)

2 distinct values for the feature ['ending_stage']


,ending_stage,nb_records,percentage
0,declined,7354,95.79
1,booked,323,4.21


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])